In [ ]:
from MalardClient.MalardClient import MalardClient
from MalardClient.DataSet import DataSet
from MalardClient.BoundingBox import BoundingBox
from MalardClient.MaskFilter import MaskFilter

%matplotlib inline
import matplotlib.pyplot as plt

import json

client = MalardClient()


In [ ]:
#Discover the parent data sets available in this environment
pds = client.getParentDataSets()

print(pds)

In [ ]:
#Query the available data sets for a given parent
datasets = client.getDataSets('cryotempo')

for ds in datasets:
    print(str(ds))


In [ ]:
#Get the bounding box for each data set
bounding_boxes = [client.boundingBox(ds) for ds in datasets]

for bb in bounding_boxes:
    print(str(bb))

In [ ]:
#get the grid cells within a bounding box
parentDs = 'cryotempo'
ds = 'GRIS_BaselineC_Q2'
region = 'greenland'

dataset = DataSet(parentDs, ds, region )
bb = client.boundingBox(dataset)

gcs = client.gridCells( dataset, bb )

print(gcs[0])

shards = client.shards(dataset, bb )

print(shards[0].shardName)

In [ ]:
#retrieve the data for the first grid cell

resultInfo = client.executeQuery( dataset, gcs[0] )

#Print the status of the request
print( resultInfo.status )

#create a dataframe from the results
df = resultInfo.to_df

#release the underlying server resources
client.releaseCacheHandle( resultInfo.resultFileName )

In [ ]:
#executeQuery polygon. Uses the shapefile passed in to determine the extent of the bounding box.
#The projection of the shapefile needs to be consistent with the data that is passed in.
#if using lat/lon for the xCol and yCol use EPSG: 4326 and (x,y) the proj4 code from the data set
from datetime import datetime

shapeFile = "/data/puma1/scratch/mtngla/chucach/Chucach-polygon_XY.shp"
extentFilter=MaskFilter(p_shapeFile = shapeFile)

alaskaDataSet = DataSet( 'mtngla', 'ADwithTDX', 'alaska' ) 

minT = datetime(2011,1,1,0,0)
maxT = datetime(2011,2,1,0,0)

chucachPolyInfo = client.executeQueryPolygon( alaskaDataSet, minT, maxT, extentFilter=extentFilter )

chucachDf = chucachPolyInfo.to_df

client.releaseCacheHandle(chucachPolyInfo.resultFileName)

In [ ]:
#Applying a shapefile with a user defined bounding box

print(chucachDf['x'].min())
print(chucachDf['x'].max())
print(chucachDf['y'].min())
print(chucachDf['y'].max())

bb = BoundingBox( -3100000, chucachDf['x'].max() , chucachDf['y'].min(), chucachDf['y'].max(), minT,maxT )

bb_chucachInfo = client.executeQuery( alaskaDataSet, bb, maskFilters=extentFilter )

bb_chucach_df = bb_chucachInfo.to_df

client.releaseCacheHandle(bb_chucachInfo.resultFileName)

In [ ]:

def simplePlot(df):
## plot with no filters
    plt.figure(figsize=(12,10))
    plt.scatter(x=df['x'], y=df['y'], c=df['elev'], marker='.', s=20, vmax=7500)
    plt.colorbar()
    plt.show()

In [ ]:
simplePlot( chucachDf )

In [ ]:
simplePlot( bb_chucach_df )

In [ ]:
#get the input swath file names from the input dataframe

file_ids = df['swathFileId'].unique()

results = client.getSwathNamesFromIds( dataset, file_ids )
print(results)

In [ ]:
filterShards = client.shardsWithinPolygon( alaskaDataSet,minT,maxT, extentFilter=extentFilter)

In [ ]:
for s in filterShards:
    print(s.shardName)

In [ ]:
shards = client.shards( alaskaDataSet,bb)

In [ ]:
for s in shards:
    print(s.shardName)

In [ ]:
###Jakobshavn bbox
minX=-262187
maxX=-93976
minY=-2386746
maxY=-2144979

minT = datetime(2011,2,1,0,0)
maxT = datetime(2011,3,1,0,0)

wkt = "POLYGON (( {} {}, {} {}, {} {}, {} {}, {} {}))".format( minX,minY,minX,maxY,maxX,maxY,maxX,minY,minX,minY )

greenland = DataSet("cryotempo","GRIS_BaselineC_Q2","greenland")

extentFilter = MaskFilter( p_wkt=wkt )

gShards = client.shardsWithinPolygon(greenland, minT, maxT, extentFilter=extentFilter)

for s in gShards:
    print(s.shardName)

In [ ]:
maskFilterIce = MaskFilter( p_shapeFile="/data/puma1/scratch/cryotempo/masks/icesheets.shp"  )


jakobshavnInfo = client.executeQueryPolygon( greenland, minT, maxT, extentFilter=extentFilter,maskFilters=maskFilterIce )

jakobshavn = jakobshavnInfo.to_df

client.releaseCacheHandle(jakobshavnInfo.resultFileName)

In [ ]:
simplePlot( jakobshavn )

In [ ]:
print(jakobshavn.columns)